In [2]:
# --- Auto-reload imported modules ---
%reload_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from word2number import w2n
import re
import torch
import sympy
from symspellpy import SymSpell, Verbosity
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from num2words import num2words

In [3]:
# ! pip install symspellpy
# ! pip install -U sentence-transformers
# ! pip install num2words

In [4]:
# -----------------------------
# 1. Initialize spell checker
# -----------------------------
# Preload a spellchecker dictionary (SymSpell requires a frequency dictionary file).
# For math explanations, you can also add custom terms like 'fraction', 'denominator', etc.
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# Load a standard dictionary (download en_US.txt frequency file from symspellpy repo)
sym_spell.load_dictionary("frequency_dictionary_en_82_765.txt", term_index=0, count_index=1)

# Add common math-specific corrections
custom_map = {
    "equak": "equal",
    "nineth": "ninth",
    "substract": "subtract",
    "denomenator": "denominator",
    "numerator": "numerator",
    "negetive": "negative",
    "mutiply": "multiply",
    "devide": "divide",
    "tree": "three",
}

# -----------------------------
# 2. Helper functions
# -----------------------------

def correct_spelling(text: str) -> str:
    """Spell-check words using SymSpell."""
    corrected = []
    for word in text.split():
        if not word.isalpha():
            corrected.append(word)
            continue
        if word in custom_map:
            corrected.append(custom_map[word])
            continue
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected.append(suggestions[0].term if suggestions else word)
    return " ".join(corrected)

def words_to_numbers(text: str) -> str:
    """Convert number words (one, two, ... thousand) into digits."""
    tokens = re.findall(r"\b[a-zA-Z\-]+\b", text.lower())
    for tok in tokens:
        try:
            num = w2n.word_to_num(tok)
            if 1 <= num <= 1000:
                text = re.sub(rf"\b{tok}\b", str(num), text, flags=re.IGNORECASE)
        except ValueError:
            continue
    return text

def normalize_fractions(text: str) -> str:
    """Convert common fraction/decimal forms into normalized fractions."""
    # Replace "one third" → "1/3" etc. (extend dictionary as needed)
    fraction_map = {
        "half": "1/2", "one half": "1/2",
        "third": "1/3", "one third": "1/3",
        "quarter": "1/4", "one quarter": "1/4",
        "two thirds": "2/3", "three quarters": "3/4"
    }
    for k, v in fraction_map.items():
        text = re.sub(rf"\b{k}\b", v, text, flags=re.IGNORECASE)

    # Replace decimals with fractions (only for common ones)
    decimal_map = {
        "0.5": "1/2", "0.25": "1/4", "0.333": "1/3", "0.75": "3/4"
    }
    for k, v in decimal_map.items():
        text = text.replace(k, v)

    # Fix "1 3" → "1/3"
    text = re.sub(r"\b(\d+)\s+(\d+)\b", r"\1/\2", text)
    return text

def normalize_operators(text: str) -> str:
    """Replace words with math operators."""
    ops = {
        "take away": "-", "minus": "-", "subtract": "-",
        "times": "*", "multiplied by": "*",
        "divide": "/", "divided by": "/", "over": "/",
        "plus": "+"
    }
    for k, v in ops.items():
        text = re.sub(rf"\b{k}\b", v, text, flags=re.IGNORECASE)
    return text

def revert_normalize_operators(text: str) -> str:
    """Revert math operators back to words (for display)."""
    ops = {
        "-": "minus",
        "*": "times",
        "/": "divided by",
        "+": "plus"
    }
    for k, v in ops.items():
        text = text.replace(k, f" {v} ")
    return re.sub(r"\s+", " ", text).strip()

def normalize_math_explanation(text: str) -> str:
    """Full pipeline: spell check, numbers, fractions, operators."""
    if not isinstance(text, str):
        return text
    
    # Step 1: spelling
    text = correct_spelling(text.lower().strip())
    # Step 2: number words → digits
    text = words_to_numbers(text)

    # # Step 3: fraction/decimal normalization
    # text = normalize_fractions(text)
    # Step 4: operator normalization
    text = normalize_operators(text)
    
    return text

# -----------------------------
# Example
# -----------------------------
examples = [
    "one third is equak to tree nineth",
    "two fifths of 10",
    "take away 5 from 8",
    "0.333 is same as 1 3"
]

for e in examples:
    print(e, " → ", normalize_math_explanation(e))

one third is equak to tree nineth  →  1 third is equal to 3 ninth
two fifths of 10  →  2 fifths of 10
take away 5 from 8  →  - 5 from 8
0.333 is same as 1 3  →  0.333 is same as 1 3


In [5]:
CLEAN_MISLABEL = "ignore"
TRAIN_CSV = "./train.csv"

In [6]:
# --- Load and Preprocess ---
def clean_mislabel_entries(train):
    print(f"Using {CLEAN_MISLABEL} for data cleaning Strat")
    question_id = 31778
    correct_answer = r"\( 6 \)"
    
    # Identify rows to fix: incorrect labels on correct answer, or true labels on incorrect answers
    rows_to_fix = []
    for idx, row in train[train['QuestionId'] == question_id].iterrows():
        is_correct_answer = row['MC_Answer'] == correct_answer
        is_true = row['Category'].startswith("True")
        if is_correct_answer and not is_true:
            rows_to_fix.append(idx)
        elif not is_correct_answer and is_true:
            rows_to_fix.append(idx)

    assert len(rows_to_fix) == 18, "Expected 18 mislabeled entries to fix, found a different number."

    if CLEAN_MISLABEL == "ignore":
        return train

    elif CLEAN_MISLABEL == "remove":
        return train.drop(index=rows_to_fix).reset_index(drop=True)

    elif CLEAN_MISLABEL == "fix":
        # Fix mislabeled rows
        for idx in rows_to_fix:
            row = train.loc[idx]
            cat = row['Category'].split("_", 1)[-1]
            prefix = "True" if row['MC_Answer'] == correct_answer else "False"
            train.at[idx, 'Category'] = f"{prefix}_{cat}"

        return train

    else:
        raise ValueError("CLEAN_MISLABEL must be one of 'do_nothing', 'remove', or 'correct'")


def load_and_preprocess_data():
    train = pd.read_csv(TRAIN_CSV)
    train = clean_mislabel_entries(train)
    
    train['Misconception'] = train['Misconception'].fillna('NA')
    train['target'] = train['Category'] + ":" + train['Misconception']

    idx = train['Category'].str.startswith("True")
    correct = (
        train[idx]
        .groupby(['QuestionId', 'MC_Answer'])
        .size()
        .reset_index(name='c')
        .sort_values('c', ascending=False)
        .drop_duplicates(['QuestionId'])
        .assign(is_correct=1)
        [['QuestionId', 'MC_Answer', 'is_correct']]
    )

    train = train.merge(correct, on=['QuestionId', 'MC_Answer'], how='left')
    train['is_correct'] = train['is_correct'].fillna(0)

    return train

In [7]:
df = load_and_preprocess_data()

display(df)

Using ignore for data cleaning Strat


,row_id,QuestionId,QuestionText,MC_Answer,StudentExplanation,Category,Misconception,target,is_correct
0,0,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),0ne third is equal to tree nineth,True_Correct,NA,True_Correct:NA,1.0
1,1,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 / 3 because 6 over 9 is 2 thirds and 1 third...,True_Correct,NA,True_Correct:NA,1.0
2,2,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),"1 3rd is half of 3 6th, so it is simplee to un...",True_Neither,NA,True_Neither:NA,1.0
3,3,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 goes into everything and 3 goes into nine,True_Neither,NA,True_Neither:NA,1.0
4,4,31772,What fraction of the shape is not shaded? Give...,\( \frac{1}{3} \),1 out of every 3 isn't coloured,True_Correct,NA,True_Correct:NA,1.0
...,...,...,...,...,...,...,...,...,...
36691,36691,109465,The probability of an event occurring is \( 0....,Unlikely,very unlikely.,False_Neither,NA,False_Neither:NA,0.0
36692,36692,109465,The probability of an event occurring is \( 0....,Unlikely,yea pretty much unlikely from what it seems.,False_Neither,NA,False_Neither:NA,0.0
36693,36693,109465,The probability of an event occurring is \( 0....,Unlikely,yeah pretty unlikelyy from what it seems.,False_Neither,NA,False_Neither:NA,0.0
36694,36694,109465,The probability of an event occurring is \( 0....,Unlikely,you have 9 other numbers to get.,False_Neither,NA,False_Neither:NA,0.0


In [8]:
# Apply normalization
tqdm.pandas()

df["ExplanationNorm"] = (
    df["StudentExplanation"]
      .astype(str)
      .str.strip()
      .str.lower()
      .progress_apply(normalize_math_explanation)
)

100%|██████████| 36696/36696 [00:02<00:00, 14768.46it/s]


In [9]:
# Find duplicated explanations
dupes = df[df["ExplanationNorm"].duplicated(keep=False)]

# Ensure a unique row_id exists
if "row_id" not in df.columns:
    df = df.reset_index(drop=True)
    df["row_id"] = df.index

# model = SentenceTransformer("all-MiniLM-L6-v2")
# model = SentenceTransformer("math-similarity/Bert-MLM_arXiv-MP-class_zbMath")
model = SentenceTransformer("google/embeddinggemma-300m")

# Compute embeddings for all explanations
embeddings = model.encode(
    df["StudentExplanation"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True
)

# Map row_id → embedding index
rowid_to_idx = {rid: i for i, rid in enumerate(df["row_id"])}

# Initialize groups from exact duplicates
grouped = dupes.groupby("ExplanationNorm")

# Precompute full similarity matrix once
sim_matrix = util.pytorch_cos_sim(embeddings, embeddings).cpu()  # N x N, keep on CPU if GPU RAM tight


Batches:   0%|          | 0/1147 [00:00<?, ?it/s]

In [10]:
# Track which row_ids are assigned
group_membership = {norm: set(g["row_id"].tolist()) for norm, g in grouped}
assigned = set().union(*group_membership.values())

iteration_num = 1
for itr in range(iteration_num):
    print(f"\nIteration {itr+1}/{iteration_num}")

    # Step 1: compute thresholds for each group
    group_thresholds = {}
    group_idxs_map = {}  # store row_idx list for each group
    for g_idx, (norm, members) in enumerate(group_membership.items()):
        member_idxs = [rowid_to_idx[rid] for rid in members]
        group_idxs_map[norm] = member_idxs
        if len(member_idxs) > 1:
            sub_sim = sim_matrix[member_idxs][:, member_idxs]
            tril_vals = sub_sim[torch.triu(torch.ones_like(sub_sim), 1) == 1]
            threshold_min = tril_vals.min().item()
            if threshold_min < 0.8:
                print(f"  Warning: Group '{norm[:40]}...' has low similarity min {threshold_min:.4f}")
                for rid in members:
                    print(f" - {rid} -> ", df.loc[df["row_id"] == rid, "StudentExplanation"].values[0])
                    print("\n")

            group_thresholds[norm] = max(0.97, min(0.97, threshold_min))
        else:
            group_thresholds[norm] = 0.97  # fallback
        # print(f"Group {g_idx}'{norm[:40]}...' with {len(members)} members, threshold: {group_thresholds[norm]:.4f}")
    
    print(f"Total groups: {len(group_membership)}, assigned rows: {len(assigned)} / {len(df)}")

    # Step 2: assign ungrouped rows (vectorized)
    unassigned = [rid for rid in df["row_id"] if rid not in assigned]
    unassigned_idxs = np.array([rowid_to_idx[rid] for rid in unassigned])

    if len(unassigned_idxs) == 0:
        break

    for norm, member_idxs in tqdm(group_idxs_map.items(), desc="Assigning new members by group"):
        sims = sim_matrix[np.ix_(unassigned_idxs, member_idxs)]  # (U, M)
        max_sims, _ = sims.max(dim=1)        # get values only
        max_sims = max_sims.cpu().numpy()

        threshold = group_thresholds[norm]
        mask = max_sims >= threshold

        for ui, ok in zip(unassigned_idxs, mask):
            if ok:
                rid = df.iloc[ui]["row_id"]
                group_membership[norm].add(rid)
                assigned.add(rid)
    # # Step 2: assign ungrouped rows
    # unassigned = [rid for rid in df["row_id"] if rid not in assigned]

    # for rid in tqdm(unassigned, desc="Assigning new members"):
    #     idx = rowid_to_idx[rid]
    #     best_group, best_score = None, -1

    #     for norm, member_idxs in group_idxs_map.items():
    #         sims = sim_matrix[idx, member_idxs]   # lookup precomputed similarities
    #         sim = sims.max().item()
    #         if sim >= group_thresholds[norm] and sim > best_score:
    #             best_score = sim
    #             best_group = norm

    #     if best_group is not None:
    #         group_membership[best_group].add(rid)
    #         assigned.add(rid)

for g_idx, (norm, members) in enumerate(group_membership.items()):
    member_idxs = [rowid_to_idx[rid] for rid in members]
    group_idxs_map[norm] = member_idxs
    if len(member_idxs) > 1:
        sub_sim = sim_matrix[member_idxs][:, member_idxs]
        tril_vals = sub_sim[torch.triu(torch.ones_like(sub_sim), 1) == 1]
        threshold_min = tril_vals.min().item()
        if threshold_min < 0.8:
            print(f"  Warning: Group '{norm[:40]}...' has low similarity min {threshold_min:.4f}")
            for rid in members:
                print(f" - {rid} -> ", df.loc[df["row_id"] == rid, "StudentExplanation"].values[0])
                print("\n")

        group_thresholds[norm] = max(0.95, min(0.95, threshold_min))
    else:
        group_thresholds[norm] = 0.95  # fallback

print(f"Total groups: {len(group_membership)}, assigned rows: {len(assigned)} / {len(df)}")


Iteration 1/1
 - 11498 ->  Because 10 add 5 is 15 and 9 take away 5 is 4


 - 11548 ->  Because ten add five is fifteen and nine minus five is four


 - 12065 ->  because 15 take away 5 is 10 and 9 take away 5 is 4


 - 11510 ->  Because 15 minus 5 is ten and 9 minus 5 is 4


Total groups: 1364, assigned rows: 3249 / 36696


Assigning new members by group: 100%|██████████| 1364/1364 [01:59<00:00, 11.45it/s] 


 - 11498 ->  Because 10 add 5 is 15 and 9 take away 5 is 4


 - 11548 ->  Because ten add five is fifteen and nine minus five is four


 - 12064 ->  because 15 take away 5 = 10 so 9 take away 5 = 4


 - 12065 ->  because 15 take away 5 is 10 and 9 take away 5 is 4


 - 11510 ->  Because 15 minus 5 is ten and 9 minus 5 is 4


Total groups: 1364, assigned rows: 4038 / 36696


In [11]:
# Sort groups by size (descending)
sorted_groups = sorted(group_membership.items(), key=lambda x: len(x[1]), reverse=True)

# Print top 3 largest groups
for g_idx, (norm, members) in enumerate(sorted_groups):
    if len(members) == 2:
        continue
    print(f"Group {g_idx}: {norm}... → {len(members)} members")
    print("Members:")
    for rid in members:
        print(" - ", rid, df.loc[df["row_id"] == rid, "StudentExplanation"].values[0])
    print("\n")


Group 0: there are 9 triangles and 3 are not shaded.... → 67 members
Members:
 -  3840 There are 9 triangles and 3 are not shaded.
 -  3841 There are 9 triangles and 3 are not shaded.
 -  3842 There are 9 triangles and 3 are not shaded.
 -  1408 There are 9 triangles and 6 of them were shaded.
 -  4739 there are 9 triangles. 3 are not shaded
 -  3973 Theree are 9 triangles and 3 are not shaded.
 -  3845 There are 9 triangles and 3 are un-shaded.
 -  3846 There are 9 triangles and 3 are unshaded in leaving 4 shaded.
 -  3976 Theree are nine triangles and three are not shaded.
 -  3849 There are 9 triangles and 3 aren't shaded
 -  1418 There are 9 triangles in all, three are not shaded.
 -  3853 There are 9 triangles and 3 none shaded.
 -  3854 There are 9 triangles and 3 not coloured in.
 -  3855 There are 9 triangles and 3 of them are blank.
 -  3858 There are 9 triangles and 3 of them are white.
 -  3865 There are 9 triangles and 6 are shaded so there are 3 that are not colored.
 -  3

In [12]:
# Convert digits in norm to words
def digits_to_words(text: str) -> str:
    return re.sub(r"\d+", lambda m: num2words(int(m.group())), text)

# Prepare group norm strings
group_norms = list(group_membership.keys())
group_norms_clean = [revert_normalize_operators(digits_to_words(norm)).strip(".") for norm in group_norms]

# Encode with SentenceTransformer
model = SentenceTransformer("google/embeddinggemma-300m")
norm_embeddings = model.encode(group_norms_clean, convert_to_tensor=True, show_progress_bar=True)

# Precompute similarity matrix
sim_matrix = util.pytorch_cos_sim(norm_embeddings, norm_embeddings).cpu()

Batches:   0%|          | 0/43 [00:00<?, ?it/s]

In [13]:
threshold = 0.99
clusters = []
visited = set()

for i, norm in enumerate(group_norms):
    if i in visited:
        continue

    cluster = {i}
    growing = True
    while growing:
        growing = False
        current_members = list(cluster)
        for j, other_norm in enumerate(group_norms):
            if j in visited or j in cluster:
                continue
            # max sim to any member in cluster
            sims = sim_matrix[j, current_members]
            if sims.max().item() >= threshold:
                cluster.add(j)
                growing = True

    # Mark visited
    for j in cluster:
        visited.add(j)
    clusters.append(cluster)

# Example: print clusters
for c_idx, cluster in enumerate(clusters):
    names = [group_norms[i] for i in cluster]
    if len(names) == 1:
        continue
    print(f"Cluster {c_idx}: {len(cluster)} norms → {names}")

print(f"Total clusters: {len(clusters)}")

Cluster 15: 2 norms → ['-8+5=-3', '-8+5= -3']
Cluster 16: 2 norms → ['/ by 5 and * by 3', '/ by 5 and * by 3.']
Cluster 18: 4 norms → ['/ by the bottom * by the top', '/ by the bottom * by the top.', '/ by the bottom and * by the top', '/ by the bottom and * by the top.']
Cluster 27: 2 norms → ['1 + 2 is 3 and 3 + 5 is 8', '1 + 2 is 3 and 3 + 5 is 8.']
Cluster 41: 2 norms → ['1/3 is thee same as 3/9.', '1/3 is the same as 3/9.']
Cluster 55: 10 norms → ['120 / 5 is 24 24 * 3 is 72', '120 / 5 is 24 24 a 3 is 72', '120 / 5 is 24 a 3 is 72', '120 / 5 is 24 and 24 * 3 is 72', '120 / 5 is 24 and 24 * by 3 is 72', '120 / 5 is 24 and 24x3 is 72.', '120 / 5 is 24, 24 * 3 is 72', '120 / 5 is 24, and 24 * 3 is 72.', '120 / 5 is 24 * 3 is 72', '120 / 5 is 24 * by 3 is 72.']
Cluster 61: 3 norms → ['120 / 5 is 24. 24 * 3 is 72', '120 / 5 is 24. 24 * 3 is 72.', '120 / 5 is 24. 24 a 3 is 72.']
Cluster 70: 2 norms → ['2 / 6 is 3', '2 / 6 is 3.']
Cluster 80: 2 norms → ['24 / 8 is 3 and 3 * 3 is 9', '24 

In [14]:
# --- manual merge map ---
# Each entry is a list of cluster ids (indices in `clusters`) that should be merged
manual_merge_map = [
    [55, 61, 246, 790],    # 120/5 → 24 then *3 → 72
    [786, 794, 795, 802, 803, 810],  # 2*12 = 24
    [263, 265, 811],       # 2*5=10, 3*5=15
    [809, 275],            # 24/2=12
    [80, 82, 276],         # 24/8=3, *3=9
    [272, 271, 806],       # two negatives → positive
    [41],                  # fraction equivalence (keep separate but could merge with other 3/9→1/3 clusters)
    [117, 197, 288, 310, 502, 1102, 1118, 1121, 1130, 1139, 1157],
    [642, 960, 962, 965],
    [653, 655],
]

# --- apply manual merge ---
merged_clusters = []
merged_ids = set()

for merge_group in manual_merge_map:
    merged_set = set()
    for cid in merge_group:
        if cid < len(clusters):
            merged_set |= clusters[cid]
            merged_ids.add(cid)
    if merged_set:
        merged_clusters.append(merged_set)

# add clusters that weren’t manually merged
for cid, cluster in enumerate(clusters):
    if cid not in merged_ids:
        merged_clusters.append(cluster)

clusters = merged_clusters

# --- print results ---
for c_idx, cluster in enumerate(clusters):
    names = [group_norms[i] for i in cluster]
    if len(names) == 1:
        continue
    print(f"Cluster {c_idx}: {len(cluster)} norms → {names}")

print(f"Total clusters: {len(clusters)}")


Cluster 0: 21 norms → ['120 / 5 is 24 24 * 3 is 72', '120 / 5 is 24 24 a 3 is 72', '120 / 5 is 24 a 3 is 72', '120 / 5 is 24 and 24 * 3 is 72', '120 / 5 is 24 and 24 * by 3 is 72', '120 / 5 is 24 and 24x3 is 72.', '120 / 5 is 24, 24 * 3 is 72', '120 / 5 is 24, and 24 * 3 is 72.', '120 / 5 is 24. 24 * 3 is 72', '120 / 5 is 24. 24 * 3 is 72.', '120 / 5 is 24. 24 a 3 is 72.', 'because 120 / 5 is 24 * 3 is 72', 'because 120 / 5 is 24 a 3 is 72', 'because 120 / 5 is 24 and 24 * 3 is 72', 'because 120 / 5 is 24 and 24x3 is 72', 'because 120 / 5 is 24 and 24x3=72', 'i think this because 120 / 5 is 24 and 24 * 3 is 72', 'i think this because 120 / 5 is 24 and 24 * by 3 is 72', 'i think this because 120 / 5 is 24 and 24 * by 3 is 72.', '120 / 5 is 24 * 3 is 72', '120 / 5 is 24 * by 3 is 72.']
Cluster 1: 15 norms → ['i think this because 12 * 2 is 24', 'i think this because 12 * 2 is 24.', 'i think this because 2*12 is 24.', 'i think this because 2x12 is 24', 'i think this because 2 * 12 equals 

In [15]:
row_to_cluster = {}

for cid, cluster in enumerate(clusters):
    for norm_idx in cluster:
        norm = group_norms[norm_idx]
        if norm not in group_membership:
            row_to_cluster[rid] = -1
        for rid in group_membership[norm]:
            row_to_cluster[rid] = cid

# Add cluster_id column to df
df["cluster_id"] = df["row_id"].map(row_to_cluster)

In [20]:
df["cluster_id"] = df["cluster_id"].fillna(-1).astype(int)

In [21]:
df.to_csv("train_with_clusters.csv", index=False)

In [19]:

# Group by explanation
grouped = df[df["cluster_id"] != -1].groupby(["cluster_id", "is_correct"])

print(len(grouped), "groups of duplicated explanations found.\n")

conficts = [ g for _, g in grouped if g["target"].nunique() > 1 ]
print(len(conficts), "of which have conflicting labels.")
print(f"{sum([len(tmp) for tmp in conficts])}/{len(df)} Entries involved in conflicts.\n")

for exp_norm, group in grouped:
    if group["target"].nunique() == 1:
        continue  # All identical, skip

    # print(group["target"].value_counts())

    # Pick one explanation (all are identical in norm form)
    qid = group["QuestionId"].iloc[0]
    qtext = group["QuestionText"].iloc[0]
    student_exp = group["StudentExplanation"].iloc[0]

    display(Latex(f"QuestionId: {qid} QuestionText: {qtext}"))
    print(group["target"].value_counts())
    # display(Latex(f"Student Explanation: {student_exp}"))

    for _, row in group.iterrows():
        mc = row["MC_Answer"]
        target = row["target"] 
        student_exp = row["StudentExplanation"]

        display(Latex(f"MC Answer: {mc}"))
        display(Latex(f"Student Explanation: {student_exp}"))
        display(Latex(f"Label: {target}"))

    print("\n\n")
        

1318 groups of duplicated explanations found.

172 of which have conflicting labels.
882/36696 Entries involved in conflicts.



<IPython.core.display.Latex object>

target
False_Misconception:Wrong_Fraction    5
False_Neither:NA                      1
Name: count, dtype: int64


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
df = pd.read_csv("train_with_clusters.csv")

row_ids_to_correct = []
cluster_ids_to_merge = []

# Group by explanation
grouped = df[df["cluster_id"] != -1].groupby(["cluster_id", "is_correct"])

print(len(grouped), "groups of duplicated explanations found.\n")

conficts = [ g for _, g in grouped if g["target"].nunique() > 1 ]
print(len(conficts), "of which have conflicting labels.")
print(f"{sum([len(tmp) for tmp in conficts])}/{len(df)} Entries involved in conflicts.\n")

for g_idx, (exp_norm, group) in enumerate(grouped):
    if group["target"].nunique() == 1:
        continue  # All identical, skip

    # print(group["target"].value_counts())

    # Pick one explanation (all are identical in norm form)
    qid = group["QuestionId"].iloc[0]
    qtext = group["QuestionText"].iloc[0]
    student_exp = group["StudentExplanation"].iloc[0]

    # display(Latex(f"Group {g_idx} - QuestionId: {qid} QuestionText: {qtext}"))
    # print(group["target"].value_counts())
    # display(Latex(f"Student Explanation: {student_exp}"))

    if g_idx in [1215,1229]:
        row_ids_to_correct += group["row_id"].tolist()

    if g_idx in [11, 135, 136, 141, 317, 337, 544, 524, 530, 558, 549, 559, 630, 750, 1206, 1214, 1243]:
        cluster_ids_to_merge += group["cluster_id"].unique().tolist()

    for _, row in group.iterrows():
        mc = row["MC_Answer"]
        target = row["target"] 
        student_exp = row["StudentExplanation"]

        # display(Latex(f" - {row['row_id']} - MC Answer: {mc}. Label: {target}"))
        # display(Latex(f"Student Explanation: {student_exp}"))
        # display(Latex(f"Label: {target}"))

    # print("\n\n")

row_ids_to_correct += [
    8847, 22073, 29669, 29670, 3217, 3225, 3229, 3298, 3305, 3844, 3846, 3859, 
    3872, 3892, 3903, 4046, 4148, 4161, 4162, 4166, 4177, 4293, 4294, 4654, 4655, 
    4689, 4711, 4712, 4731, 4765, 1377, 2184, 2191, 1733, 1408, 30157, 9546, 
    20796, 3129, 3152, 3151, 4630, 4753, 4688
]

print("Rows to correct:", row_ids_to_correct)
print("Clusters to merge:", cluster_ids_to_merge)



df.loc[df["row_id"].isin(row_ids_to_correct), "cluster_id"] = -1
df.loc[df["cluster_id"].isin(cluster_ids_to_merge), "cluster_id"] = 7
df.to_csv("train_with_clusters_manual_checked.csv", index=False)